In [1]:
import GlobalLocalTransformer
import vgg
from GlobalLocalTransformer import *
import torch as nn
import torch.nn as nn

import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import TrainTestGLT
from TrainTestGLT import *
import imp
imp.reload(GlobalLocalTransformer)
imp.reload(vgg)
imp.reload(TrainTestGLT)

<module 'TrainTestGLT' from '/media/dataanalyticlab/Drive2/MANSOOR/Neuroimaging_Project/Code/Brain_Age_Estimation/DL_based_BAE/GlobalLocalTransformer/TrainTestGLT.py'>

In [2]:
src = "/media/dataanalyticlab/Drive2/MANSOOR/Dataset/Test/"
X = np.load(src + "VBM_OpenBHB.npy")
Y = pd.read_csv(src + "SubInfoOpenBHB.csv")
print("Data matrix loaded... shape: ", X.shape )
labels = Y.loc[:,"age"]
x = X[:,:,:,60:65,0]
print(type(x))

Data matrix loaded... shape:  (20, 121, 145, 121, 1)
<class 'numpy.ndarray'>


In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(x,labels, test_size=0.2, random_state=42)

X_train = torch.from_numpy(X_train)
X_train = X_train.permute(0,3,1,2)
print(X_train.shape)


X_test = torch.from_numpy(X_test)
X_test = X_test.permute(0,3,1,2)
print(X_test.shape)



Y_train = torch.from_numpy(np.array(Y_train))
Y_test = torch.from_numpy(np.array(Y_test))

print(Y_train.shape, Y_test.shape)


torch.Size([16, 5, 121, 145])
torch.Size([4, 5, 121, 145])
torch.Size([16]) torch.Size([4])


In [11]:
train_dataloader = DataLoader(list(zip(X_train, Y_train)), batch_size=5, shuffle=True)
test_dataloader = DataLoader(X_test, batch_size=4, shuffle=True)

In [12]:
regression_model = nn.Linear(13, 1)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
iters, losses, train_acc, val_acc = [], [], [], []

regression_model = regression_model.double()
# output_tensors = output_tensors.double()
# torch.autograd.set_detect_anomaly(True)

# Loop through the number of epochs
for epoch in range(5):
    total_loss = 0  # Initialize loss for the entire epoch
    total_mae = 0.0   # Initialize the total MAE for the entire epoch
    optimizer.zero_grad()  # a clean up step for PyTorch

    # Training loop
    regression_model.train()
    zlist = model(X_train)             # forward pass
    output_tensors = torch.cat(zlist, dim=1)
    # Loop through the batches and update the gradients

    # for inputs, labels in train_dataloader:


    for i in range(len(X_train)):
        final_output = regression_model(output_tensors[i])
        loss = criterion(final_output, Y_train[i])  # Compute the loss for each sample
        total_loss += loss  # Accumulate the loss for the entire batch

    avg_loss = total_loss / len(X_train)  # Calculate the average loss for the batch
    print("Epoch:", epoch, "Loss:", avg_loss)

    avg_loss.backward(retain_graph=True)  # Backward pass (compute parameter updates)
    print("backward pass completed...")
    optimizer.step()
    print("Epoch", epoch, "completed")

    # Validation loop 
    regression_model.eval()  # Set the model to evaluation model
    total_samples = 0
    zlist = model(X_test)             # forward pass
    output_tensors = torch.cat(zlist, dim=1)

    with torch.no_grad():
        for i in range(len(X_test)):
            outputs = regression_model(output_tensors[i])
            mae = torch.mean(torch.abs(outputs - Y_test[i]))
            total_mae += mae.item()
            total_samples += 1

        avg_mae = total_mae / total_samples  # Calculate the average MAE for the epoch
        print("Epoch:", epoch, "Mean Absolute Error (MAE):", avg_mae)

/home/dataanalyticlab/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 0 Loss: tensor(19.3501, dtype=torch.float64, grad_fn=<DivBackward0>)
backward pass completed...
Epoch 0 completed
Epoch: 0 Mean Absolute Error (MAE): 25.32518721515599
Epoch: 1 Loss: tensor(17.7104, dtype=torch.float64, grad_fn=<DivBackward0>)
backward pass completed...
Epoch 1 completed
Epoch: 1 Mean Absolute Error (MAE): 24.600474847878335
Epoch: 2 Loss: tensor(16.1519, dtype=torch.float64, grad_fn=<DivBackward0>)
backward pass completed...
Epoch 2 completed
Epoch: 2 Mean Absolute Error (MAE): 23.937253197037744
Epoch: 3 Loss: tensor(14.5911, dtype=torch.float64, grad_fn=<DivBackward0>)
backward pass completed...
Epoch 3 completed
Epoch: 3 Mean Absolute Error (MAE): 23.603286593955918
Epoch: 4 Loss: tensor(13.2688, dtype=torch.float64, grad_fn=<DivBackward0>)
backward pass completed...
Epoch 4 completed
Epoch: 4 Mean Absolute Error (MAE): 23.419284455853585


In [273]:
def get_accuracy(model, X_set, Y_set):
   
    correct = 0
    total = 0
    for i in range(len(X_set)):
        output = model(X_set) # We don't need to run F.softmax
        pred = output[0].max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(Y_set.view_as(pred)).sum().item()
        total += X_set[i].shape[0]
    return correct / total

In [ ]:
model = GlobalLocalBrainAge(5,
                        patch_size=64,
                        step=32,
                        nblock=6,
                        backbone='vgg8')

model = model.double()
acc = train(model, X_train, Y_train)
# zlist = model(x)
# for z in zlist:
    # print(z.shape)
    
# Assuming `output_tensors` is a list of 'n' tensors, each of size mx1, representing predictions for 'n' patches.
# You can concatenate them along the patch dimension (n) to form a single tensor of size m x n x 1.
# output_tensors = torch.cat(zlist, dim=1)

# Now, compute the mean along the patch dimension (n) to obtain a single tensor of size mx1 for each sample.
# aggregated_predictions = torch.mean(output_tensors, dim=1)
# aggregated_predictions

In [237]:
prob = F.softmax(zlist[0][6], dim=0)
print(prob)
# print(sum(prob[0]))

tensor([1.], dtype=torch.float64, grad_fn=<SoftmaxBackward0>)
